In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os
import pandas as pd
from numeric2content_function.machine_data import machine2content
from numeric2content_function.machine_health_data import machineHealth2content
from numeric2content_function.component_list_data import componentList2content
from numeric2content_function.components_health_data import allComponentsHealth2content
from numeric2content_function.all_models_health_data import allModelsHealth2content

In [10]:
def pipeline_numeric2content(raw_data_dir, prepared_data_dir):

    # อ่านรายการโรงงานจากไฟล์ CSV
    file_name = "plant_list.csv"
    plant_list_file = os.path.join(raw_data_dir, file_name)
    if os.path.isfile(plant_list_file):
        plant_df = pd.read_csv(plant_list_file)
        # สร้างโฟลเดอร์สำหรับเก็บข้อมูลที่ประมวลผลเเล้ว
        save_to_dir = os.path.join(prepared_data_dir)
        os.makedirs(save_to_dir, exist_ok=True)
        # บันทึกรายการโรงงานที่จะประมวลผล
        save_plant_lis_full_file_path = os.path.join(save_to_dir, file_name)
        plant_df.to_csv(save_plant_lis_full_file_path, index=False)
    else:
        print(f":: Failed ❌ - File not found: {plant_list_file}")
        return  # ออกจากฟังก์ชันถ้าไม่พบไฟล์

    # วนลูปผ่านแต่ละโรงงาน
    for _, plant_row in plant_df.iterrows():
        plant_tag = plant_row["PLANT_TAG"]
        plant_name = plant_row["PLANT_NAME"]

        # อ่านรายการเครื่องจักรของแต่ละโรงงาน
        file_name = "machine_list.csv"
        machine_list_file = os.path.join(raw_data_dir, plant_tag, file_name)
        if os.path.isfile(machine_list_file):
            machine_df = pd.read_csv(machine_list_file)

            # บันทึกรายการเครื่องจักรที่จะประมวลผล
            save_to_dir = os.path.join(prepared_data_dir, plant_tag)
            os.makedirs(save_to_dir, exist_ok=True)
            save_machineList_full_file_path = os.path.join(save_to_dir, file_name)
            machine_df[
                ["MACHINE_ID", "MACHINE_TAG", "MACHINE_NAME", "DESCRIPTION"]
            ].to_csv(save_machineList_full_file_path, index=False)
        else:
            print(f":: Failed ❌ - File not found: {machine_list_file}")
            break

        # วนลูปผ่านแต่ละเครื่องจักร
        for idx, machine_row in machine_df.iterrows():
            machine_tag = machine_row["MACHINE_TAG"]
            machine_name = machine_row["MACHINE_NAME"]
            print("\n" + "=" * 100)
            print(
                f"#{idx+1} Processing content for {plant_name} (TAG: {plant_tag}) - {machine_name} (TAG: {machine_tag})"
            )
            print("=" * 100)

            # สร้างเนื้อหาข้อมูลเครื่องจักร
            print(f"\n>> Create: Machine content - File processing: {file_name}")
            machines_content = machine2content(plant_row, machine_row)
            print(f":: Complete ✔️")

            # สร้างเนื้อหาสุขภาพเครื่องจักร
            file_name = "all_machines_health.csv"
            print(f"\n>> Create: Machine health content - File processing: {file_name}")
            all_machines_health_file = os.path.join(raw_data_dir, plant_tag, file_name)
            if os.path.isfile(all_machines_health_file):
                all_machines_health_df = pd.read_csv(all_machines_health_file)
                machine_health_df = all_machines_health_df[
                    all_machines_health_df["MACHINE_TAG"] == machine_tag
                ]
                machinesHealth_content = machineHealth2content(
                    plant_row, machine_health_df
                )
                print(f":: Complete ✔️")
            else:
                print(f":: Failed ❌- File not found: {all_machines_health_file}")
                machinesHealth_content = ""

            # สร้างเนื้อหารายการส่วนประกอบ
            file_name = "component_list.csv"
            print(f"\n>> Create: Component list content - File processing: {file_name}")
            component_list_file = os.path.join(
                raw_data_dir, plant_tag, machine_tag, file_name
            )
            if os.path.isfile(component_list_file):
                component_list_df = pd.read_csv(component_list_file)
                componentList_content = componentList2content(
                    plant_row, machine_row, component_list_df
                )
                print(f":: Complete ✔️")
            else:
                print(f":: Failed ❌ - File not found: {component_list_file}")
                componentList_content = ""

            # สร้างเนื้อหาสุขภาพส่วนประกอบทั้งหมด
            file_name = "all_components_health.csv"
            print(
                f"\n>> Create: ALL Component health content - File processing: {file_name}"
            )
            all_components_health_file = os.path.join(
                raw_data_dir, plant_tag, machine_tag, file_name
            )
            if os.path.isfile(all_components_health_file):
                all_components_health_df = pd.read_csv(all_components_health_file)
                allComponentsHealth_content = allComponentsHealth2content(
                    plant_row, machine_row, all_components_health_df
                )
                print(f":: Complete ✔️")
            else:
                print(f":: Failed ❌ - File not found: {all_components_health_file}")
                allComponentsHealth_content = ""

            # สร้างเนื้อหาสุขภาพโมเดลทั้งหมด
            file_name = "all_models_health.csv"
            print(
                f"\n>> Create: ALL Model health content - File processing: {file_name}"
            )
            all_models_health_file = os.path.join(
                raw_data_dir, plant_tag, machine_tag, file_name
            )
            if os.path.isfile(all_models_health_file):
                all_models_health_df = pd.read_csv(all_models_health_file)
                allModelsHealth_content = allModelsHealth2content(
                    plant_row, machine_row, component_list_df, all_models_health_df
                )
                print(f":: Complete ✔️")
            else:
                print(f":: Failed ❌ - File not found: {all_models_health_file}")
                allModelsHealth_content = ""

            # รวมเนื้อหาทั้งหมดของเครื่องจักร
            print(f"\n>> Create: Corpus of Machine content")
            corpusMachine_content = "\n".join(
                [
                    machines_content,
                    machinesHealth_content,
                    componentList_content,
                    allComponentsHealth_content,
                    allModelsHealth_content,
                ]
            )
            print(f":: Complete ✔️")
            print(f"\nDisplay\n\n{corpusMachine_content}")

            # บันทึกเนื้อหาที่ประมวลผลแล้ว
            print(f"\n>> Create: Save to prepared content")
            file_name = f"{plant_tag}_{machine_tag}_corpus.txt"

            save_to_dir = os.path.join(prepared_data_dir, plant_tag, machine_tag)
            os.makedirs(save_to_dir, exist_ok=True)

            corpusMachine_full_file_path = os.path.join(save_to_dir, file_name)

            # เขียนเนื้อหาลงไฟล์
            with open(corpusMachine_full_file_path, "w") as f:
                f.write(corpusMachine_content)

            print(f":: Complete ✔️ - File name: {file_name} - Successfully saved to: {corpusMachine_full_file_path} ")

            print("=" * 100)
            print("Complete ALL")
            print("=" * 100)

In [11]:
# กำหนด root directory หลักของโปรเจค
ROOT_DIRECTORY = "D:\Data_sci_internship\Exploring Generative AI for Predictive Maintenance Applications"

# กำหนดชื่อโฟลเดอร์ย่อยต่างๆ
PROJECT_DIRECTORY = "predictive-maintenance-chatbot"    # โฟลเดอร์โปรเจค
DATA_ROOT_DIRECTORY = "data"                            # โฟลเดอร์หลักสำหรับเก็บข้อมูล
RAW_DATA_DIRECTORY = "raw_data"                         # โฟลเดอร์สำหรับข้อมูลดิบ
PREPARED_DATA_DIRECTORY = "prepared_data"               # โฟลเดอร์สำหรับข้อมูลที่ประมวลผลแล้ว

# สร้าง path เต็มสำหรับโฟลเดอร์ข้อมูลดิบ
raw_data_dir = os.path.join(
    ROOT_DIRECTORY, PROJECT_DIRECTORY, DATA_ROOT_DIRECTORY, RAW_DATA_DIRECTORY
)

# สร้าง path เต็มสำหรับโฟลเดอร์ข้อมูลที่ประมวลผลแล้ว
prepared_data_dir = os.path.join(
    ROOT_DIRECTORY, PROJECT_DIRECTORY, DATA_ROOT_DIRECTORY, PREPARED_DATA_DIRECTORY
)

# เรียกใช้ฟังก์ชัน pipeline เพื่อประมวลผลข้อมูล
pipeline_numeric2content(raw_data_dir, prepared_data_dir)


#1 Processing content for Natural Gas Processing Plant (TAG: PLANT_01) - Sale Gas Compressor (TAG: COMP_SG01)

>> Create: Machine content - File processing: machine_list.csv
:: Complete ✔️

>> Create: Machine health content - File processing: all_machines_health.csv
:: Complete ✔️

>> Create: Component list content - File processing: component_list.csv
:: Complete ✔️

>> Create: ALL Component health content - File processing: all_components_health.csv
:: Complete ✔️

>> Create: ALL Model health content - File processing: all_models_health.csv
:: Complete ✔️

>> Create: Corpus of Machine content
:: Complete ✔️

Display

[Section]MACHINE DATA[Section]
* The Sale Gas Compressor (Tag: COMP_SG01) in the Natural Gas Processing Plant is currently RUNNING. It is monitored by the associated sensor SIA005A.PV, with a stop threshold set at 5500. The machine consists of 8 components and 167 models.
[Section]MACHINE HEALTH DATA[Section]
* The Sale Gas Compressor (Tag: COMP_SG01) in the Natural Gas